# Doenças Cronicas não Transmissiveis

Este estudo visa analisar os dados de óbitos por doenças crônicas não transmissíveis entre 2013 até 2023

Perguntas Chave
1. Quais são as taxas de mortalidade por diferentes tipos de DCNTs (ex: diabetes, doenças cardíacas, câncer) em diferentes regiões?
2. Existem padrões sazonais ou temporais nas taxas de mortalidade por DCNTs?
3. Quais são os grupos demográficos (ex: idade, gênero, etnia) mais afetados por diferentes DCNTs?
4. Como fatores socioeconômicos (ex: renda, nível de educação, acesso a cuidados de saúde) estão correlacionados com as taxas de mortalidade por DCNTs?
5. Existem disparidades geográficas significativas nas taxas de mortalidade por DCNTs?
6. Como as taxas de mortalidade por DCNTs mudaram ao longo do tempo?
7. Quais são os principais fatores de risco associados às DCNTs, de acordo com os dados disponíveis?
8. Há diferenças significativas nas taxas de mortalidade por DCNTs entre áreas urbanas e rurais?

Objetivos da Pesquisa
* Identificar Padrões: Descobrir padrões e tendências nas taxas de mortalidade por DCNTs, tanto ao longo do tempo quanto entre diferentes grupos demográficos e geográficos.
* Informar Políticas de Saúde: Fornecer insights baseados em dados para ajudar na formulação de políticas de saúde pública que visem reduzir as taxas de mortalidade por DCNTs.
* Melhorar a Alocação de Recursos: Identificar áreas e grupos demográficos que necessitam de mais recursos e atenção para reduzir as disparidades nas taxas de mortalidade.
* Promover a Prevenção: Identificar fatores de risco e promover estratégias de prevenção para diminuir a incidência de DCNTs.
* Avaliar Intervenções: Avaliar a eficácia de intervenções e programas de saúde existentes voltados para a prevenção e tratamento de DCNTs.
* Aumentar a Conscientização: Informar e educar a população sobre os riscos e a prevenção das DCNTs com base em evidências empíricas.


---

In [ ]:
%pip install plotly

## importando bibliotecas

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.express as px


---
## limpando os dados

In [ ]:
datasets = {}

# read every file in dados/
for file in os.listdir('dados/'):
    if file.endswith('.csv'):
        datasets[file] = pd.read_csv('dados/' + file, delimiter=';')

Já que temos muitas amostras e relativamente poucos valores nulos, podemos apenas removê-los, já que não farão muita falta nos dados

In [ ]:
for name, df in datasets.items():
    df.dropna(inplace=True)
    

---
## concatenando os datasets

juntando todos os dados 

In [ ]:
#concatenate all datasets
df = pd.concat(datasets.values())
df.shape


---
## analisando valores

In [ ]:
df.info()

analisando valores unicos 

In [ ]:
df.co_cid_causa_basica.unique().shape

In [ ]:
#remove rows where tp_raca_cor is "Amarela" or "Indigena"
df = df[df.tp_raca_cor != 'Amarela']
df = df[df.tp_raca_cor != 'Indígena']

---
## visão geral dos dados

In [ ]:
df.loc[df['co_cid_causa_basica'] == 'E149']

quantidade de obitos por etnia

In [ ]:
# df.tp_raca_cor.value_counts().plot(kind='barh')
#plotly plot pie chart
fig = px.pie(df, names='tp_raca_cor', title='Raca/Cor')
#fig size make it squared
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
)

fig.show()



distribuição da idade no óbito

In [ ]:
# df.nu_idade.plot(kind='hist', bins=20)
#plotly plot histogram
fig = px.histogram(df, x='nu_idade', title='Idade')
fig.update_layout(
    autosize=False,
    width=700,
    height=500,
)
fig.show()


homens vs mulheres

In [ ]:
#show percentages
# plt.pie(df.sg_sexo.value_counts(), labels=['Masculino', 'Feminino'], autopct='%1.1f%%')
#plotly plot pie chart
fig = px.pie(df, names='sg_sexo', title='Sexo')
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
)
fig.show()

homens vs mulheres agrupados por raça

In [ ]:
men_vs_woman_by_race = df.groupby(['tp_raca_cor', 'sg_sexo']).size().unstack()
men_vs_woman_by_race.plot(kind='bar', stacked=False)

#plotly plot bar chart


---
## análises centradas


nesta seção faremos analises mais especificas focando mais nas causas do óbito e comparando raças e sexos 

Objetivos:
- Visualizar a relação entre homens e mulheres de cada raça e analisar as causas de óbito mais comuns para cada classe 
- Rastrear essas relações ao longo do tempo, pegando esses dados a cada ano e colocando em um grafico temporal 

Verificando os códigos no nosso dataset

In [ ]:
df.capitulo_cid_causa_basica.unique()

In [ ]:
grouped_cid_raca = df.groupby(['capitulo_cid_causa_basica', 'tp_raca_cor']).size().unstack()
grouped_cid_raca.plot(kind='barh', stacked=False)


Analisando as mortes por cid ao longo dos anos

In [ ]:
#organizar por ano de óbito 
df['dt_obito'] = pd.to_datetime(df['dt_obito'], dayfirst=True)
df['year'] = df['dt_obito'].dt.year

In [ ]:
#morte de homens por ano para cada cid e raça 
mortes_homens = df.groupby(['year', 'capitulo_cid_causa_basica', 'tp_raca_cor', 'sg_sexo']).size().reset_index(name='count')
fig = px.line(
    mortes_homens,
    x='year',
    y='count',
    color='capitulo_cid_causa_basica',
    facet_col='tp_raca_cor',
    facet_row='sg_sexo',
    markers=True
)

fig.update_layout(width=1200, height=600)
fig.show()